In [1]:
#import depencies
import pandas as pd

In [2]:
# import csv files
students_data = "02-Homework_04-Pandas_Instructions_PyCitySchools_Resources_students_complete.csv"
school_data = "02-Homework_04-Pandas_Instructions_PyCitySchools_Resources_schools_complete.csv"
student_data_df = pd.read_csv(students_data)
school_data_df = pd.read_csv(school_data)
student_data_df.head()
school_data_df
#student_data_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [3]:
#district summary

total_schools = school_data_df["School ID"].count()
total_students = school_data_df["size"].sum()
total_budget = school_data_df["budget"].sum()
average_math = student_data_df["math_score"].sum() / total_students
average_reading = student_data_df["reading_score"].sum() / total_students
percent_math = student_data_df.loc[student_data_df["math_score"] >=70]
percent_math = percent_math["math_score"].count() / total_students * 100 
percent_reading = student_data_df.loc[student_data_df["reading_score"] >=70]
percent_reading = percent_reading["reading_score"].count() / total_students * 100 
percent_both = student_data_df.loc[(student_data_df["math_score"] >=70) & (student_data_df["reading_score"] >=70)]
percent_both = percent_both["student_name"].count() / total_students * 100
data = ({"Total Schools": [total_schools],
      "Total Students": [total_students],
      "Total Budget": [total_budget],
      "Average Math Score": [average_math],
      "Average Reading Score": [average_reading],
      "% Passing Math": [percent_math],
      "% Passing Reading": [percent_reading],
      "% Overall Passing": [percent_both]})
district_summary_df = pd.DataFrame(data)
formated_summary = district_summary_df.copy()
formated_summary["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
formated_summary["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
formated_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [4]:
#school summary
grouped_students = student_data_df.groupby(["school_name"]).sum()
grouped_students = pd.DataFrame(grouped_students)
grouped_students = grouped_students.reset_index(drop=False)
grouped_students = grouped_students.rename(columns={"school_name": " "})

number_students = student_data_df["school_name"].value_counts()
summary_df = pd.DataFrame(number_students)
summary_df = summary_df.reset_index(drop=False)

summary_df = summary_df.rename(columns={ "school_name": "Total Students",
                                         "index": "school_name"})
mergered_summary_df = pd.merge(summary_df, school_data_df, on="school_name", how="inner" )
mergered_summary_df

del mergered_summary_df["size"] 
del mergered_summary_df["School ID"]
mergered_summary2_df = mergered_summary_df[["school_name", "type", "Total Students", "budget"]]
mergered_summary3_df = mergered_summary2_df.rename(columns={"type": "School Type", 
                                                            "budget": "Total School Budget",
                                                            "school_name": " "})
merged4 = pd.merge(grouped_students, mergered_summary3_df, on=" ", how="outer")
merged4["Per Student Budget"] = merged4.iloc[0:15, 6] / merged4.iloc[0:15, 5]
merged4["Average Math Score"] = merged4.iloc[0:15, 3] / merged4.iloc[0:15, 5]
merged4["Average Reading Score"] = merged4.iloc[0:15, 2] / merged4.iloc[0:15, 5]

del merged4["Student ID"]
del merged4["reading_score"]
del merged4["math_score"]

grade_sort = student_data_df.loc[(student_data_df["math_score"] >=70) & (student_data_df["reading_score"] >=70)]
grade_show = pd.DataFrame(grade_sort)
total_grouped = grade_show.groupby(["school_name"]).count()
total_grouped = total_grouped.rename(columns={"reading_score": "total", "math_score": "total_math"})

math_sort = student_data_df.loc[(student_data_df["math_score"] >=70)] 
math_show = pd.DataFrame(math_sort)
math_grouped= math_show.groupby(["school_name"]).count()
del math_grouped["reading_score"]

reading_sort = student_data_df.loc[(student_data_df["reading_score"] >=70)] 
reading_show = pd.DataFrame(reading_sort)
reading_grouped= reading_show.groupby(["school_name"]).count()
del reading_grouped["math_score"]

merge_score_by_school = pd.merge(math_grouped, reading_grouped, on="school_name", how="inner")
merge_score_all = pd.merge(merge_score_by_school, total_grouped, on='school_name', how="inner")

del merge_score_all["Student ID_x"]
del merge_score_all["student_name_x"]
del merge_score_all["gender_x"]
del merge_score_all["grade_x"]
del merge_score_all["Student ID_y"]
del merge_score_all["student_name_y"]
del merge_score_all["gender_y"]
del merge_score_all["grade_y"]
del merge_score_all["Student ID"]
del merge_score_all["student_name"]
del merge_score_all["gender"]
del merge_score_all["grade"]
del merge_score_all["total_math"]

merge_score_all = merge_score_all.reset_index(drop=False)
merge_score_all = merge_score_all.rename(columns={"school_name": " "})

last_columns = pd.merge(merge_score_all, merged4, on=" ", how="inner")

last_columns["% Passing Math"] = last_columns.iloc[0:15, 1] / last_columns.iloc[0:15, 5] *100
last_columns["% Passing Reading"] = last_columns.iloc[0:15, 2] / last_columns.iloc[0:15, 5] *100
last_columns["% Overall Passing"] = last_columns.iloc[0:15, 3] / last_columns.iloc[0:15, 5] *100
del last_columns["math_score"]
del last_columns["reading_score"]
del last_columns["total"]

last_columns = last_columns.sort_values([" "], ascending=True)
last_columns = last_columns.set_index(' ')
last_columns

formated_final_table = last_columns.copy()
formated_final_table["Total School Budget"] = last_columns["Total School Budget"].map("${:,.2f}".format) 
formated_final_table["Per Student Budget"] = last_columns["Per Student Budget"].map("${:,.2f}".format)
formated_final_table

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [5]:
# Top Performing Schools
best_performing = formated_final_table.sort_values(["% Overall Passing"], ascending=False)
best_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [7]:
# Bottom Perfroming Schools
worst_performing = formated_final_table.sort_values(["% Overall Passing"], ascending=True)
worst_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [8]:
#Math Scores by Grade

ninth_data = student_data_df.loc[(student_data_df["grade"] =="9th")]
ninth_sort = pd.DataFrame(ninth_data)
tenth_data = student_data_df.loc[(student_data_df["grade"] =="10th")]
tenth_sort = pd.DataFrame(tenth_data)
eleven_data = student_data_df.loc[(student_data_df["grade"] =="11th")]
eleven_sort = pd.DataFrame(eleven_data)
twelve_data = student_data_df.loc[(student_data_df["grade"] =="12th")]
twelve_sort = pd.DataFrame(twelve_data)

#9th
ninth_bailey = ninth_sort.loc[(ninth_sort["school_name"] == "Bailey High School")]
ninth_bailey_df = pd.DataFrame(ninth_bailey)
bailey_ninth_average_m = ninth_bailey_df["math_score"].sum()/ ninth_bailey_df["math_score"].count()

ninth_cabrera = ninth_sort.loc[(ninth_sort["school_name"] == "Cabrera High School")]
ninth_cabrera_df = pd.DataFrame(ninth_cabrera)
cabrera_ninth_average_m = ninth_cabrera_df["math_score"].sum()/ ninth_cabrera_df["math_score"].count()

figueroa_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Figueroa High School")]
figueroa_ninth_df = pd.DataFrame(figueroa_ninth)
figueroa_ninth_average_m = figueroa_ninth_df["math_score"].sum()/ figueroa_ninth_df["math_score"].count()

ford_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Ford High School")]
ford_ninth_df = pd.DataFrame(ford_ninth)
ford_ninth_average_m = ford_ninth_df["math_score"].sum()/ ford_ninth_df["math_score"].count()

griffin_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Griffin High School")]
griffin_ninth_df = pd.DataFrame(griffin_ninth)
griffin_ninth_average_m = griffin_ninth_df["math_score"].sum()/ griffin_ninth_df["math_score"].count()

hernan_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Hernandez High School")]
hernan_ninth_df = pd.DataFrame(hernan_ninth)
hernan_ninth_average_m = hernan_ninth_df["math_score"].sum()/ hernan_ninth_df["math_score"].count()

hold_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Holden High School")]
hold_ninth_df = pd.DataFrame(hold_ninth)
hold_ninth_average_m = hold_ninth_df["math_score"].sum()/ hold_ninth_df["math_score"].count()


huang_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Huang High School")]
huang_ninth_df = pd.DataFrame(huang_ninth)
huang__ninth_average_m = huang_ninth_df["math_score"].sum()/ huang_ninth_df["math_score"].count()

johnson_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Johnson High School")]
johnson_ninth_df = pd.DataFrame(johnson_ninth)
johnson_ninth_average_m = johnson_ninth_df["math_score"].sum()/ johnson_ninth_df["math_score"].count()


Pena_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Pena High School")]
Pena_ninth_df = pd.DataFrame(Pena_ninth)
Pena_ninth_average_m = Pena_ninth_df["math_score"].sum()/ Pena_ninth_df["math_score"].count()

Rodriguez_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Rodriguez High School")]
Rodriguez_ninth_df = pd.DataFrame(Rodriguez_ninth)
Rodriguez_ninth_average_m = Rodriguez_ninth_df["math_score"].sum()/ Rodriguez_ninth_df["math_score"].count()

Shelton_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Shelton High School")]
Shelton_ninth_df = pd.DataFrame(Shelton_ninth)
Shelton_ninth_average_m = Shelton_ninth_df["math_score"].sum()/ Shelton_ninth_df["math_score"].count()

Thomas_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Thomas High School")]
Thomas_ninth_df = pd.DataFrame(Thomas_ninth)
Thomas_ninth_average_m = Thomas_ninth_df["math_score"].sum()/ Thomas_ninth_df["math_score"].count()

Wilson_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Wilson High School")]
Wilson_ninth_df = pd.DataFrame(Wilson_ninth)
Wilson_ninth_average_m = Wilson_ninth_df["math_score"].sum()/ Wilson_ninth_df["math_score"].count()

Wright_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Wright High School")]
Wright_ninth_df = pd.DataFrame(Wright_ninth)
Wright_ninth_average_m = Wright_ninth_df["math_score"].sum()/ Wright_ninth_df["math_score"].count()

#10th
ten_bailey = tenth_sort.loc[(tenth_sort["school_name"] == "Bailey High School")]
ten_bailey_df = pd.DataFrame(ten_bailey)
bailey_ten_average_m = ten_bailey_df["math_score"].sum()/ ten_bailey_df["math_score"].count()

ten_cabrera = tenth_sort.loc[(tenth_sort["school_name"] == "Cabrera High School")]
ten_cabrera_df = pd.DataFrame(ten_cabrera)
cabrera_ten_average_m = ten_cabrera_df["math_score"].sum()/ ten_cabrera_df["math_score"].count()

figueroa_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Figueroa High School")]
figueroa_ten_df = pd.DataFrame(figueroa_ten)
figueroa_ten_average_m = figueroa_ten_df["math_score"].sum()/ figueroa_ten_df["math_score"].count()

ford_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Ford High School")]
ford_ten_df = pd.DataFrame(ford_ten)
ford_ten_average_m = ford_ten_df["math_score"].sum()/ ford_ten_df["math_score"].count()

griffin_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Griffin High School")]
griffin_ten_df = pd.DataFrame(griffin_ten)
griffin_ten_average_m = griffin_ten_df["math_score"].sum()/ griffin_ten_df["math_score"].count()

hernan_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Hernandez High School")]
hernan_ten_df = pd.DataFrame(hernan_ten)
hernan_ten_average_m = hernan_ten_df["math_score"].sum()/ hernan_ten_df["math_score"].count()

hold_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Holden High School")]
hold_ten_df = pd.DataFrame(hold_ten)
hold_ten_average_m = hold_ten_df["math_score"].sum()/ hold_ten_df["math_score"].count()


huang_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Huang High School")]
huang_ten_df = pd.DataFrame(huang_ten)
huang_ten_average_m = huang_ten_df["math_score"].sum()/ huang_ten_df["math_score"].count()

johnson_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Johnson High School")]
johnson_ten_df = pd.DataFrame(johnson_ten)
johnson_ten_average_m = johnson_ten_df["math_score"].sum()/ johnson_ten_df["math_score"].count()


Pena_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Pena High School")]
Pena_ten_df = pd.DataFrame(Pena_ten)
Pena_ten_average_m = Pena_ten_df["math_score"].sum()/ Pena_ten_df["math_score"].count()

Rodriguez_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Rodriguez High School")]
Rodriguez_ten_df = pd.DataFrame(Rodriguez_ten)
Rodriguez_ten_average_m = Rodriguez_ten_df["math_score"].sum()/ Rodriguez_ten_df["math_score"].count()

Shelton_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Shelton High School")]
Shelton_ten_df = pd.DataFrame(Shelton_ninth)
Shelton_ten_average_m = Shelton_ten_df["math_score"].sum()/ Shelton_ten_df["math_score"].count()

Thomas_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Thomas High School")]
Thomas_ten_df = pd.DataFrame(Thomas_ten)
Thomas_ten_average_m = Thomas_ten_df["math_score"].sum()/ Thomas_ten_df["math_score"].count()

Wilson_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Wilson High School")]
Wilson_ten_df = pd.DataFrame(Wilson_ten)
Wilson_ten_average_m = Wilson_ten_df["math_score"].sum()/ Wilson_ten_df["math_score"].count()

Wright_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Wright High School")]
Wright_ten_df = pd.DataFrame(Wright_ten)
Wright_ten_average_m = Wright_ten_df["math_score"].sum()/ Wright_ten_df["math_score"].count()

#11th
eleven_bailey = eleven_sort.loc[(eleven_sort["school_name"] == "Bailey High School")]
eleven_bailey_df = pd.DataFrame(eleven_bailey)
bailey_eleven_average_m = eleven_bailey_df["math_score"].sum()/ eleven_bailey_df["math_score"].count()

eleven_cabrera = eleven_sort.loc[(eleven_sort["school_name"] == "Cabrera High School")]
eleven_cabrera_df = pd.DataFrame(eleven_cabrera)
cabrera_eleven_average_m = eleven_cabrera_df["math_score"].sum()/ eleven_cabrera_df["math_score"].count()

figueroa_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Figueroa High School")]
figueroa_eleven_df = pd.DataFrame(figueroa_eleven)
figueroa_eleven_average_m = figueroa_eleven_df["math_score"].sum()/ figueroa_eleven_df["math_score"].count()

ford_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Ford High School")]
ford_eleven_df = pd.DataFrame(ford_eleven)
ford_eleven_average_m = ford_eleven_df["math_score"].sum()/ ford_eleven_df["math_score"].count()

griffin_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Griffin High School")]
griffin_eleven_df = pd.DataFrame(griffin_eleven)
griffin_eleven_average_m = griffin_eleven_df["math_score"].sum()/ griffin_eleven_df["math_score"].count()

hernan_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Hernandez High School")]
hernan_eleven_df = pd.DataFrame(hernan_eleven)
hernan_eleven_average_m = hernan_eleven_df["math_score"].sum()/ hernan_eleven_df["math_score"].count()

hold_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Holden High School")]
hold_eleven_df = pd.DataFrame(hold_eleven)
hold_eleven_average_m = hold_eleven_df["math_score"].sum()/ hold_eleven_df["math_score"].count()


huang_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Huang High School")]
huang_eleven_df = pd.DataFrame(huang_eleven)
huang_eleven_average_m = huang_eleven_df["math_score"].sum()/ huang_eleven_df["math_score"].count()

johnson_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Johnson High School")]
johnson_eleven_df = pd.DataFrame(johnson_eleven)
johnson_eleven_average_m = johnson_eleven_df["math_score"].sum()/ johnson_eleven_df["math_score"].count()

Pena_eleven = ninth_sort.loc[(ninth_sort["school_name"] == "Pena High School")]
Pena_eleven_df = pd.DataFrame(Pena_eleven)
Pena_eleven_average_m = Pena_eleven_df["math_score"].sum()/ Pena_eleven_df["math_score"].count()

Rodriguez_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Rodriguez High School")]
Rodriguez_eleven_df = pd.DataFrame(Rodriguez_eleven)
Rodriguez_eleven_average_m = Rodriguez_eleven_df["math_score"].sum()/ Rodriguez_eleven_df["math_score"].count()

Shelton_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Shelton High School")]
Shelton_eleven_df = pd.DataFrame(Shelton_eleven)
Shelton_eleven_average_m = Shelton_eleven_df["math_score"].sum()/ Shelton_ninth_df["math_score"].count()

Thomas_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Thomas High School")]
Thomas_eleven_df = pd.DataFrame(Thomas_eleven)
Thomas_eleven_average_m = Thomas_eleven_df["math_score"].sum()/ Thomas_eleven_df["math_score"].count()

Wilson_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Wilson High School")]
Wilson_eleven_df = pd.DataFrame(Wilson_eleven)
Wilson_eleven_average_m = Wilson_eleven_df["math_score"].sum()/ Wilson_eleven_df["math_score"].count()

Wright_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Wright High School")]
Wright_eleven_df = pd.DataFrame(Wright_eleven)
Wright_eleven_average_m = Wright_eleven_df["math_score"].sum()/ Wright_eleven_df["math_score"].count()


#12th
twelve_bailey = twelve_sort.loc[(twelve_sort["school_name"] == "Bailey High School")]
twelve_bailey_df = pd.DataFrame(twelve_bailey)
bailey_twelve_average_m = twelve_bailey_df["math_score"].sum()/ twelve_bailey_df["math_score"].count()

twelve_cabrera = twelve_sort.loc[(twelve_sort["school_name"] == "Cabrera High School")]
twelve_cabrera_df = pd.DataFrame(twelve_cabrera)
cabrera_twelve_average_m = twelve_cabrera_df["math_score"].sum()/ twelve_cabrera_df["math_score"].count()

figueroa_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Figueroa High School")]
figueroa_twelve_df = pd.DataFrame(figueroa_twelve)
figueroa_twelve_average_m = figueroa_twelve_df["math_score"].sum()/ figueroa_twelve_df["math_score"].count()

ford_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Ford High School")]
ford_twelve_df = pd.DataFrame(ford_twelve)
ford_twelve_average_m = ford_twelve_df["math_score"].sum()/ ford_twelve_df["math_score"].count()

griffin_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Griffin High School")]
griffin_twelve_df = pd.DataFrame(griffin_twelve)
griffin_twelve_average_m = griffin_twelve_df["math_score"].sum()/ griffin_twelve_df["math_score"].count()

hernan_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Hernandez High School")]
hernan_twelve_df = pd.DataFrame(hernan_twelve)
hernan_twelve_average_m = hernan_twelve_df["math_score"].sum()/ hernan_twelve_df["math_score"].count()

hold_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Holden High School")]
hold_twelve_df = pd.DataFrame(hold_twelve)
hold_twelve_average_m = hold_twelve_df["math_score"].sum()/ hold_twelve_df["math_score"].count()


huang_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Huang High School")]
huang_twelve_df = pd.DataFrame(huang_twelve)
huang_twelve_average_m = huang_twelve_df["math_score"].sum()/ huang_twelve_df["math_score"].count()

johnson_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Johnson High School")]
johnson_twelve_df = pd.DataFrame(johnson_twelve)
johnson_twelve_average_m = johnson_twelve_df["math_score"].sum()/ johnson_twelve_df["math_score"].count()

Pena_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Pena High School")]
Pena_twelve_df = pd.DataFrame(Pena_twelve)
Pena_twelve_average_m = Pena_twelve_df["math_score"].sum()/ Pena_twelve_df["math_score"].count()

Rodriguez_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Rodriguez High School")]
Rodriguez_twelve_df = pd.DataFrame(Rodriguez_twelve)
Rodriguez_twelve_average_m = Rodriguez_twelve_df["math_score"].sum()/ Rodriguez_twelve_df["math_score"].count()

Shelton_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Shelton High School")]
Shelton_twelve_df = pd.DataFrame(Shelton_twelve)
Shelton_twelve_average_m = Shelton_twelve_df["math_score"].sum()/ Shelton_twelve_df["math_score"].count()

Thomas_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Thomas High School")]
Thomas_twelve_df = pd.DataFrame(Thomas_twelve)
Thomas_twelve_average_m = Thomas_twelve_df["math_score"].sum()/ Thomas_twelve_df["math_score"].count()

Wilson_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Wilson High School")]
Wilson_twelve_df = pd.DataFrame(Wilson_twelve)
Wilson_twelve_average_m = Wilson_twelve_df["math_score"].sum()/ Wilson_twelve_df["math_score"].count()

Wright_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Wright High School")]
Wright_twelve_df = pd.DataFrame(Wright_twelve)
Wright_twelve_average_m = Wright_twelve_df["math_score"].sum()/ Wright_twelve_df["math_score"].count()



grade_score_data_math = {" ": ["Bailey High School", "Cabrera High School", "Figueroa High School", "Ford High School",
                          "Griffin High School", "Hernandez High School", "Holden High School", "Huang High School",
                          "Johnson High School", "Pena High School", "Rodriguez High School", "Shelton High School",
                          "Thomas High School", "Wilson High School", "Wright High School"],
                    
                   "9th": [bailey_ninth_average_m, cabrera_ninth_average_m, figueroa_ninth_average_m,
                           ford_ninth_average_m, griffin_ninth_average_m, hernan_ninth_average_m,
                           hold_ninth_average_m, huang__ninth_average_m, johnson_ninth_average_m,
                           Pena_ninth_average_m, Rodriguez_ninth_average_m, Shelton_ninth_average_m,
                           Thomas_ninth_average_m, Wilson_ninth_average_m, Wright_ninth_average_m],
                    
                   "10th": [bailey_ten_average_m, cabrera_ten_average_m, figueroa_ten_average_m,
                           ford_ten_average_m, griffin_ten_average_m, hernan_ten_average_m,
                           hold_ten_average_m, huang_ten_average_m, johnson_ten_average_m,
                           Pena_ten_average_m, Rodriguez_ten_average_m, Shelton_ten_average_m,
                           Thomas_ten_average_m, Wilson_ten_average_m, Wright_ten_average_m],
                    
                    "11th": [bailey_eleven_average_m, cabrera_eleven_average_m, figueroa_eleven_average_m,
                           ford_eleven_average_m, griffin_eleven_average_m, hernan_eleven_average_m,
                           hold_eleven_average_m, huang_eleven_average_m, johnson_eleven_average_m,
                           Pena_eleven_average_m, Rodriguez_eleven_average_m, Shelton_eleven_average_m,
                           Thomas_eleven_average_m, Wilson_eleven_average_m, Wright_eleven_average_m],
                    
                    "12th": [bailey_twelve_average_m, cabrera_twelve_average_m, figueroa_twelve_average_m,
                           ford_twelve_average_m, griffin_twelve_average_m, hernan_twelve_average_m,
                           hold_twelve_average_m, huang_twelve_average_m, johnson_twelve_average_m,
                           Pena_twelve_average_m, Rodriguez_twelve_average_m, Shelton_twelve_average_m,
                           Thomas_twelve_average_m, Wilson_twelve_average_m, Wright_twelve_average_m]}

                   

final_frame_m = pd.DataFrame(grade_score_data_math)

final_frame_math = final_frame_m.set_index(" ")
final_frame_math

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [9]:
#Reading Score by Grade

ninth_data = student_data_df.loc[(student_data_df["grade"] =="9th")]
ninth_sort = pd.DataFrame(ninth_data)
tenth_data = student_data_df.loc[(student_data_df["grade"] =="10th")]
tenth_sort = pd.DataFrame(tenth_data)
eleven_data = student_data_df.loc[(student_data_df["grade"] =="11th")]
eleven_sort = pd.DataFrame(eleven_data)
twelve_data = student_data_df.loc[(student_data_df["grade"] =="12th")]
twelve_sort = pd.DataFrame(twelve_data)

#9th
ninth_bailey = ninth_sort.loc[(ninth_sort["school_name"] == "Bailey High School")]
ninth_bailey_df_r = pd.DataFrame(ninth_bailey)
bailey_ninth_average_r = ninth_bailey_df_r['reading_score'].sum()/ ninth_bailey_df_r['reading_score'].count()

ninth_cabrera = ninth_sort.loc[(ninth_sort["school_name"] == "Cabrera High School")]
ninth_cabrera_df_r = pd.DataFrame(ninth_cabrera)
cabrera_ninth_average_r = ninth_cabrera_df_r['reading_score'].sum()/ ninth_cabrera_df_r['reading_score'].count()

figueroa_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Figueroa High School")]
figueroa_ninth_df_r = pd.DataFrame(figueroa_ninth)
figueroa_ninth_average_r = figueroa_ninth_df_r['reading_score'].sum()/ figueroa_ninth_df_r['reading_score'].count()

ford_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Ford High School")]
ford_ninth_df_r = pd.DataFrame(ford_ninth)
ford_ninth_average_r = ford_ninth_df_r['reading_score'].sum()/ ford_ninth_df_r['reading_score'].count()

griffin_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Griffin High School")]
griffin_ninth_df_r = pd.DataFrame(griffin_ninth)
griffin_ninth_average_r = griffin_ninth_df_r['reading_score'].sum()/ griffin_ninth_df_r['reading_score'].count()

hernan_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Hernandez High School")]
hernan_ninth_df_r = pd.DataFrame(hernan_ninth)
hernan_ninth_average_r = hernan_ninth_df_r['reading_score'].sum()/ hernan_ninth_df_r['reading_score'].count()

hold_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Holden High School")]
hold_ninth_df_r = pd.DataFrame(hold_ninth)
hold_ninth_average_r = hold_ninth_df_r['reading_score'].sum()/ hold_ninth_df_r['reading_score'].count()


huang_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Huang High School")]
huang_ninth_df_r = pd.DataFrame(huang_ninth)
huang__ninth_average_r = huang_ninth_df_r['reading_score'].sum()/ huang_ninth_df_r['reading_score'].count()

johnson_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Johnson High School")]
johnson_ninth_df_r = pd.DataFrame(johnson_ninth)
johnson_ninth_average_r = johnson_ninth_df_r['reading_score'].sum()/ johnson_ninth_df_r['reading_score'].count()


Pena_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Pena High School")]
Pena_ninth_df_r = pd.DataFrame(Pena_ninth)
Pena_ninth_average_r = Pena_ninth_df_r['reading_score'].sum()/ Pena_ninth_df_r['reading_score'].count()

Rodriguez_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Rodriguez High School")]
Rodriguez_ninth_df_r = pd.DataFrame(Rodriguez_ninth)
Rodriguez_ninth_average_r = Rodriguez_ninth_df_r['reading_score'].sum()/ Rodriguez_ninth_df_r['reading_score'].count()

Shelton_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Shelton High School")]
Shelton_ninth_df_r = pd.DataFrame(Shelton_ninth)
Shelton_ninth_average_r = Shelton_ninth_df_r['reading_score'].sum()/ Shelton_ninth_df_r['reading_score'].count()

Thomas_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Thomas High School")]
Thomas_ninth_df_r = pd.DataFrame(Thomas_ninth)
Thomas_ninth_average_r = Thomas_ninth_df_r['reading_score'].sum()/ Thomas_ninth_df_r['reading_score'].count()

Wilson_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Wilson High School")]
Wilson_ninth_df_r = pd.DataFrame(Wilson_ninth)
Wilson_ninth_average_r = Wilson_ninth_df_r['reading_score'].sum()/ Wilson_ninth_df_r['reading_score'].count()

Wright_ninth = ninth_sort.loc[(ninth_sort["school_name"] == "Wright High School")]
Wright_ninth_df_r = pd.DataFrame(Wright_ninth)
Wright_ninth_average_r = Wright_ninth_df_r['reading_score'].sum()/ Wright_ninth_df_r['reading_score'].count()

#10th
ten_bailey = tenth_sort.loc[(tenth_sort["school_name"] == "Bailey High School")]
ten_bailey_df_r = pd.DataFrame(ten_bailey)
bailey_ten_average_r = ten_bailey_df_r['reading_score'].sum()/ ten_bailey_df_r['reading_score'].count()

ten_cabrera = tenth_sort.loc[(tenth_sort["school_name"] == "Cabrera High School")]
ten_cabrera_df_r = pd.DataFrame(ten_cabrera)
cabrera_ten_average_r = ten_cabrera_df_r['reading_score'].sum()/ ten_cabrera_df_r['reading_score'].count()

figueroa_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Figueroa High School")]
figueroa_ten_df_r = pd.DataFrame(figueroa_ten)
figueroa_ten_average_r = figueroa_ten_df_r['reading_score'].sum()/ figueroa_ten_df_r['reading_score'].count()

ford_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Ford High School")]
ford_ten_df_r = pd.DataFrame(ford_ten)
ford_ten_average_r = ford_ten_df_r['reading_score'].sum()/ ford_ten_df_r['reading_score'].count()

griffin_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Griffin High School")]
griffin_ten_df_r = pd.DataFrame(griffin_ten)
griffin_ten_average_r = griffin_ten_df_r['reading_score'].sum()/ griffin_ten_df_r['reading_score'].count()

hernan_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Hernandez High School")]
hernan_ten_df_r = pd.DataFrame(hernan_ten)
hernan_ten_average_r = hernan_ten_df_r['reading_score'].sum()/ hernan_ten_df_r['reading_score'].count()

hold_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Holden High School")]
hold_ten_df_r = pd.DataFrame(hold_ten)
hold_ten_average_r = hold_ten_df_r['reading_score'].sum()/ hold_ten_df_r['reading_score'].count()


huang_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Huang High School")]
huang_ten_df_r = pd.DataFrame(huang_ten)
huang_ten_average_r = huang_ten_df_r['reading_score'].sum()/ huang_ten_df_r['reading_score'].count()

johnson_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Johnson High School")]
johnson_ten_df_r = pd.DataFrame(johnson_ten)
johnson_ten_average_r = johnson_ten_df_r['reading_score'].sum()/ johnson_ten_df_r['reading_score'].count()


Pena_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Pena High School")]
Pena_ten_df_r = pd.DataFrame(Pena_ten)
Pena_ten_average_r = Pena_ten_df_r['reading_score'].sum()/ Pena_ten_df_r['reading_score'].count()

Rodriguez_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Rodriguez High School")]
Rodriguez_ten_df_r = pd.DataFrame(Rodriguez_ten)
Rodriguez_ten_average_r = Rodriguez_ten_df_r['reading_score'].sum()/ Rodriguez_ten_df_r['reading_score'].count()

Shelton_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Shelton High School")]
Shelton_ten_df_r = pd.DataFrame(Shelton_ninth)
Shelton_ten_average_r = Shelton_ten_df_r['reading_score'].sum()/ Shelton_ten_df_r['reading_score'].count()

Thomas_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Thomas High School")]
Thomas_ten_df_r = pd.DataFrame(Thomas_ten)
Thomas_ten_average_r = Thomas_ten_df_r['reading_score'].sum()/ Thomas_ten_df_r['reading_score'].count()

Wilson_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Wilson High School")]
Wilson_ten_df_r = pd.DataFrame(Wilson_ten)
Wilson_ten_average_r = Wilson_ten_df_r['reading_score'].sum()/ Wilson_ten_df_r['reading_score'].count()

Wright_ten = tenth_sort.loc[(tenth_sort["school_name"] == "Wright High School")]
Wright_ten_df_r = pd.DataFrame(Wright_ten)
Wright_ten_average_r = Wright_ten_df_r['reading_score'].sum()/ Wright_ten_df_r['reading_score'].count()

#11th
eleven_bailey = eleven_sort.loc[(eleven_sort["school_name"] == "Bailey High School")]
eleven_bailey_df_r = pd.DataFrame(eleven_bailey)
bailey_eleven_average_r = eleven_bailey_df_r['reading_score'].sum()/ eleven_bailey_df_r['reading_score'].count()

eleven_cabrera = eleven_sort.loc[(eleven_sort["school_name"] == "Cabrera High School")]
eleven_cabrera_df_r = pd.DataFrame(eleven_cabrera)
cabrera_eleven_average_r = eleven_cabrera_df_r['reading_score'].sum()/ eleven_cabrera_df_r['reading_score'].count()

figueroa_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Figueroa High School")]
figueroa_eleven_df_r = pd.DataFrame(figueroa_eleven)
figueroa_eleven_average_r = figueroa_eleven_df_r['reading_score'].sum()/ figueroa_eleven_df_r['reading_score'].count()

ford_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Ford High School")]
ford_eleven_df_r = pd.DataFrame(ford_eleven)
ford_eleven_average_r = ford_eleven_df_r['reading_score'].sum()/ ford_eleven_df_r['reading_score'].count()

griffin_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Griffin High School")]
griffin_eleven_df_r = pd.DataFrame(griffin_eleven)
griffin_eleven_average_r = griffin_eleven_df_r['reading_score'].sum()/ griffin_eleven_df_r['reading_score'].count()

hernan_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Hernandez High School")]
hernan_eleven_df_r = pd.DataFrame(hernan_eleven)
hernan_eleven_average_r = hernan_eleven_df_r['reading_score'].sum()/ hernan_eleven_df_r['reading_score'].count()

hold_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Holden High School")]
hold_eleven_df_r = pd.DataFrame(hold_eleven)
hold_eleven_average_r = hold_eleven_df_r['reading_score'].sum()/ hold_eleven_df_r['reading_score'].count()


huang_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Huang High School")]
huang_eleven_df_r = pd.DataFrame(huang_eleven)
huang_eleven_average_r = huang_eleven_df_r['reading_score'].sum()/ huang_eleven_df_r['reading_score'].count()

johnson_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Johnson High School")]
johnson_eleven_df_r = pd.DataFrame(johnson_eleven)
johnson_eleven_average_r = johnson_eleven_df_r['reading_score'].sum()/ johnson_eleven_df_r['reading_score'].count()

Pena_eleven = ninth_sort.loc[(ninth_sort["school_name"] == "Pena High School")]
Pena_eleven_df_r = pd.DataFrame(Pena_eleven)
Pena_eleven_average_r = Pena_eleven_df_r['reading_score'].sum()/ Pena_eleven_df_r['reading_score'].count()

Rodriguez_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Rodriguez High School")]
Rodriguez_eleven_df_r = pd.DataFrame(Rodriguez_eleven)
Rodriguez_eleven_average_r = Rodriguez_eleven_df_r['reading_score'].sum()/ Rodriguez_eleven_df_r['reading_score'].count()

Shelton_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Shelton High School")]
Shelton_eleven_df_r = pd.DataFrame(Shelton_eleven)
Shelton_eleven_average_r = Shelton_eleven_df_r['reading_score'].sum()/ Shelton_ninth_df_r['reading_score'].count()

Thomas_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Thomas High School")]
Thomas_eleven_df_r = pd.DataFrame(Thomas_eleven)
Thomas_eleven_average_r = Thomas_eleven_df_r['reading_score'].sum()/ Thomas_eleven_df_r['reading_score'].count()

Wilson_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Wilson High School")]
Wilson_eleven_df_r = pd.DataFrame(Wilson_eleven)
Wilson_eleven_average_r = Wilson_eleven_df_r['reading_score'].sum()/ Wilson_eleven_df_r['reading_score'].count()

Wright_eleven = eleven_sort.loc[(eleven_sort["school_name"] == "Wright High School")]
Wright_eleven_df_r = pd.DataFrame(Wright_eleven)
Wright_eleven_average_r = Wright_eleven_df_r['reading_score'].sum()/ Wright_eleven_df_r['reading_score'].count()


#12th
twelve_bailey = twelve_sort.loc[(twelve_sort["school_name"] == "Bailey High School")]
twelve_bailey_df_r = pd.DataFrame(twelve_bailey)
bailey_twelve_average_r = twelve_bailey_df_r['reading_score'].sum()/ twelve_bailey_df_r['reading_score'].count()

twelve_cabrera = twelve_sort.loc[(twelve_sort["school_name"] == "Cabrera High School")]
twelve_cabrera_df_r = pd.DataFrame(twelve_cabrera)
cabrera_twelve_average_r = twelve_cabrera_df_r['reading_score'].sum()/ twelve_cabrera_df_r['reading_score'].count()

figueroa_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Figueroa High School")]
figueroa_twelve_df_r = pd.DataFrame(figueroa_twelve)
figueroa_twelve_average_r = figueroa_twelve_df_r['reading_score'].sum()/ figueroa_twelve_df_r['reading_score'].count()

ford_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Ford High School")]
ford_twelve_df_r = pd.DataFrame(ford_twelve)
ford_twelve_average_r = ford_twelve_df_r['reading_score'].sum()/ ford_twelve_df_r['reading_score'].count()

griffin_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Griffin High School")]
griffin_twelve_df_r = pd.DataFrame(griffin_twelve)
griffin_twelve_average_r = griffin_twelve_df_r['reading_score'].sum()/ griffin_twelve_df_r['reading_score'].count()

hernan_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Hernandez High School")]
hernan_twelve_df_r = pd.DataFrame(hernan_twelve)
hernan_twelve_average_r = hernan_twelve_df_r['reading_score'].sum()/ hernan_twelve_df_r['reading_score'].count()

hold_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Holden High School")]
hold_twelve_df_r = pd.DataFrame(hold_twelve)
hold_twelve_average_r = hold_twelve_df_r['reading_score'].sum()/ hold_twelve_df_r['reading_score'].count()


huang_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Huang High School")]
huang_twelve_df_r = pd.DataFrame(huang_twelve)
huang_twelve_average_r = huang_twelve_df_r['reading_score'].sum()/ huang_twelve_df_r['reading_score'].count()

johnson_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Johnson High School")]
johnson_twelve_df_r = pd.DataFrame(johnson_twelve)
johnson_twelve_average_r = johnson_twelve_df_r['reading_score'].sum()/ johnson_twelve_df_r['reading_score'].count()

Pena_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Pena High School")]
Pena_twelve_df_r = pd.DataFrame(Pena_twelve)
Pena_twelve_average_r = Pena_twelve_df_r['reading_score'].sum()/ Pena_twelve_df_r['reading_score'].count()

Rodriguez_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Rodriguez High School")]
Rodriguez_twelve_df_r = pd.DataFrame(Rodriguez_twelve)
Rodriguez_twelve_average_r = Rodriguez_twelve_df_r['reading_score'].sum()/ Rodriguez_twelve_df_r['reading_score'].count()

Shelton_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Shelton High School")]
Shelton_twelve_df_r = pd.DataFrame(Shelton_twelve)
Shelton_twelve_average_r = Shelton_twelve_df_r['reading_score'].sum()/ Shelton_twelve_df_r['reading_score'].count()

Thomas_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Thomas High School")]
Thomas_twelve_df_r = pd.DataFrame(Thomas_twelve)
Thomas_twelve_average_r = Thomas_twelve_df_r['reading_score'].sum()/ Thomas_twelve_df_r['reading_score'].count()

Wilson_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Wilson High School")]
Wilson_twelve_df_r = pd.DataFrame(Wilson_twelve)
Wilson_twelve_average_r = Wilson_twelve_df_r['reading_score'].sum()/ Wilson_twelve_df_r['reading_score'].count()

Wright_twelve = twelve_sort.loc[(twelve_sort["school_name"] == "Wright High School")]
Wright_twelve_df_r = pd.DataFrame(Wright_twelve)
Wright_twelve_average_r = Wright_twelve_df_r['reading_score'].sum()/ Wright_twelve_df_r['reading_score'].count()



grade_score_data_reading = {" ": ["Bailey High School", "Cabrera High School", "Figueroa High School", "Ford High School",
                          "Griffin High School", "Hernandez High School", "Holden High School", "Huang High School",
                          "Johnson High School", "Pena High School", "Rodriguez High School", "Shelton High School",
                          "Thomas High School", "Wilson High School", "Wright High School"],
                    
                   "9th": [bailey_ninth_average_r, cabrera_ninth_average_r, figueroa_ninth_average_r,
                           ford_ninth_average_r, griffin_ninth_average_r, hernan_ninth_average_r,
                           hold_ninth_average_r, huang__ninth_average_r, johnson_ninth_average_r,
                           Pena_ninth_average_r, Rodriguez_ninth_average_r, Shelton_ninth_average_r,
                           Thomas_ninth_average_r, Wilson_ninth_average_r, Wright_ninth_average_r],
                    
                   "10th": [bailey_ten_average_r, cabrera_ten_average_r, figueroa_ten_average_r,
                           ford_ten_average_r, griffin_ten_average_r, hernan_ten_average_r,
                           hold_ten_average_r, huang_ten_average_r, johnson_ten_average_r,
                           Pena_ten_average_r, Rodriguez_ten_average_r, Shelton_ten_average_r,
                           Thomas_ten_average_r, Wilson_ten_average_r, Wright_ten_average_r],
                    
                    "11th": [bailey_eleven_average_r, cabrera_eleven_average_r, figueroa_eleven_average_r,
                           ford_eleven_average_r, griffin_eleven_average_r, hernan_eleven_average_r,
                           hold_eleven_average_r, huang_eleven_average_r, johnson_eleven_average_r,
                           Pena_eleven_average_r, Rodriguez_eleven_average_r, Shelton_eleven_average_r,
                           Thomas_eleven_average_r, Wilson_eleven_average_r, Wright_eleven_average_r],
                    
                    "12th": [bailey_twelve_average_r, cabrera_twelve_average_r, figueroa_twelve_average_r,
                           ford_twelve_average_r, griffin_twelve_average_r, hernan_twelve_average_r,
                           hold_twelve_average_r, huang_twelve_average_r, johnson_twelve_average_r,
                           Pena_twelve_average_r, Rodriguez_twelve_average_r, Shelton_twelve_average_r,
                           Thomas_twelve_average_r, Wilson_twelve_average_r, Wright_twelve_average_r]}

                   

final_frame2 = pd.DataFrame(grade_score_data_reading)

final_frame_reading = final_frame2.set_index(" ")
final_frame_reading




,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [10]:
# Scores by School Spending 
#set up bins 
new_columns = last_columns.copy()
student_spend_df = pd.DataFrame(new_columns)
spending_bins = [0, 584, 629, 644, 675]
spending_labels = ["<584", "585-629", "630-644", "645-675"]

#create data frame 
student_spend_df["Spending Ranges (Per Student)"] = pd.cut(student_spend_df["Per Student Budget"], spending_bins, labels=spending_labels)
student_spend_df
per_student_df = student_spend_df.groupby("Spending Ranges (Per Student)")


per_student_df[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<584,83.455399,83.933814,93.460096,96.610877,90.369459
585-629,81.899826,83.155286,87.133538,92.718205,81.418596
630-644,78.518855,81.624473,73.484209,84.391793,62.857656
645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [11]:
# Scored by School Size
#set up bins 
school_size_df = pd.DataFrame(new_columns)
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#create data frame 
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], size_bins, labels=size_labels)
total_student_df = school_size_df.groupby("School Size")


total_student_df[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [12]:
#Scores by School Type

school_type_df = pd.DataFrame(new_columns)
grouped_type = school_type_df.groupby("School Type")

charter_group = grouped_type.get_group("Charter")
district_group = grouped_type.get_group("District")
cdf = charter_group[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()
ddf = district_group[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()
type_data = {"School Type": ["Charter", "District"],
                "Average Math Score": [cdf["Average Math Score"], ddf["Average Math Score"]],
                "Average Reading Score": [cdf["Average Reading Score"], ddf["Average Reading Score"]],
                "% Passing Math": [cdf["% Passing Math"], ddf["% Passing Math"]],
                "% Passing Reading": [cdf["% Passing Reading"], ddf["% Passing Reading"]], 
                "% Overall Passing": [cdf["% Overall Passing"], ddf["% Overall Passing"]]}
type_df = pd.DataFrame(type_data)
type_df

formatted_type = type_df.set_index('School Type')
formatted_type



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
